In [ ]:
# Scripts to do:
# - Refresher
# - Mapper

In [ ]:
###############################################################################################################
##################################### Initialisation ##########################################################
###############################################################################################################

In [2]:
import os
from pymongo import MongoClient
import bycon
import pandas as pd
import time, base36

# Prepare mongodb tools
client = MongoClient()
db = client.progenetix
bs = db.biosamples

In [3]:
df = pd.read_csv('../temp/mappingfile.tsv', sep = '\t')

In [16]:
df.keys()

Index(['biosample_id', 'variant_id', 'callset_id', 'chromosome', 'start',
       'end', 'strand', 'reference_bases', 'alternate_bases', 'hgvsc',
       'variant_classification', 'variant_type', 'hgvsp', 'hgvsp_short',
       'aliquot_id', 'reference_id', 'case_id', 'sample_id'],
      dtype='object')

In [14]:
dd_filtered = df.loc[df['variant_type'] == 'SNP', ['hgvsc', 'reference_bases',
                                                   'alternate_bases', 'start', 'end']].copy()
INS = df.loc[df['variant_type'] == 'INS', ['start', 'end', 'reference_bases', 'alternate_bases']].copy()
# Drop rows with missing values in HGVSC
dd_filtered.dropna(subset=['hgvsc'], inplace=True)

# Extract the last character from HGVSC and assign it to a new column 'HGVSC_ref_base'
dd_filtered['HGVSC_ref_base'] = dd_filtered['hgvsc'].str.split('>').str[0].str[-1]
dd_filtered['HGVSC_alt_base'] = dd_filtered['hgvsc'].str.split('>').str[1]


# Reset the index if necessary
dd_filtered.reset_index(drop=True, inplace=True)

compare = dd_filtered[['hgvsc', 'start', 'end', 'HGVSC_ref_base', 'reference_bases', 'HGVSC_alt_base', 'alternate_bases']]
issues = compare[compare['reference_bases'] != compare['HGVSC_ref_base']]
correct = compare[compare['reference_bases'] == compare['HGVSC_ref_base']]

In [15]:
correct

,hgvsc,start,end,HGVSC_ref_base,reference_bases,HGVSC_alt_base,alternate_bases
1,c.103G>C,226064454,226064454,G,G,C,C
2,c.658A>T,26279162,26279162,A,A,T,T
3,c.1380C>A,31266476,31266476,C,C,A,A
4,c.3484C>A,98303765,98303765,C,C,A,A
5,c.2510T>A,124764127,124764127,T,T,A,A
...,...,...,...,...,...,...,...
2424419,c.315G>T,106612395,106612395,G,G,T,T
2424420,c.813G>A,114906851,114906851,G,G,A,A
2424422,c.165T>A,139537086,139537086,T,T,A,A
2424423,c.56G>T,153536303,153536303,G,G,T,T
